# Simulation Results Analyzer

This notebook analyzes existing saved simulation results from the `run_simulation_notebook.ipynb` (or its script equivalent).

**Functionality:**
- Loads per-run metrics (`.json` files) from `OUTPUT_DIR_RUN_METRICS_JSON`.
- Loads posterior summaries (mean, quantiles as `.npz` files) for the first MC run of each scenario from `OUTPUT_DIR_POSTERIOR_SUMMARIES`.
- Regenerates true data (`sim_data`) for the first MC run of each scenario using stored seeds to provide context for plots.
- Generates and saves individual time-series plots for the first MC run of each scenario (to `OUTPUT_DIR_PLOTS`).
- Generates and saves a combined 4x3 grid plot of these time-series (to `OUTPUT_DIR_PLOTS`).
- Generates and saves summary box plots for all evaluation metrics across all MC runs (to `OUTPUT_DIR_PLOTS`).
- Aggregates metrics and produces LaTeX summary tables (to `OUTPUT_DIR_TABLES`).

**Prerequisites:**
1. The simulation runner notebook/script must have completed, and its output files must be available in the directories specified in `config.py`.
2. All helper Python modules (`config.py`, `data_generation.py`, etc.) must be in the same directory or accessible.

## 1. Imports and Configuration

In [1]:
import pandas as pd
import numpy as np
import os
import time
import json
from tqdm.auto import tqdm
import importlib

# Import and force-reload all your custom modules to ensure the latest versions are used
import config; importlib.reload(config)
import data_generation; importlib.reload(data_generation)
import benchmarks; importlib.reload(benchmarks)
import evaluation; importlib.reload(evaluation)
import plotting; importlib.reload(plotting)
import tables; importlib.reload(tables)
import results_io; importlib.reload(results_io)

print("All modules have been imported and reloaded.")
print("Reading configuration and output paths from config.py")

All modules have been imported and reloaded.
Reading configuration and output paths from config.py


## 2. Main Analysis Function

In [2]:
def sanitize_metrics_dataframe(df):
    """
    Cleans a DataFrame of metrics by converting list-like values to scalars.
    This is a safeguard against malformed metric files from older runs.
    """
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if the column contains list-like objects and flatten them
            if df[col].notna().any() and isinstance(df[col].dropna().iloc[0], list):
                df[col] = df[col].apply(
                    lambda x: x[0] if isinstance(x, list) and len(x) == 1 else (np.nan if isinstance(x, list) and len(x) == 0 else x)
                ).astype(float, errors='ignore')
    return df

def prepare_aggregated_plot_data(results_df_all):
    """
    Aggregates time-series results from all valid MC runs for the summary plot.
    This version prepares data for sCFR-O (as requested for plotting) and the benchmarks.
    """
    aggregated_plot_data_list = []
    study_global_seed = config.GLOBAL_BASE_SEED

    for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Aggregating Plot Data")):
        scenario_id = scenario_config_dict["id"]
        scenario_base_seed = study_global_seed + (scenario_idx * config.NUM_MONTE_CARLO_RUNS * 1000)
        
        sim_data_true = data_generation.simulate_scenario_data(scenario_config_dict, run_seed=scenario_base_seed)
        T_analyze = config.T_ANALYSIS_LENGTH

        scen_df_valid = results_df_all[(results_df_all["scenario_id"] == scenario_id) & (results_df_all["error"].isin([None, "None"]))]
        if scen_df_valid.empty: continue
        
        # Stacks for collecting time-series data from all valid runs
        sCFR_means, sCFR_lowers, sCFR_uppers, sCFR_cf_means, sCFR_cf_lowers, sCFR_cf_uppers = [], [], [], [], [], []
        cCFR_means, cCFR_lowers, cCFR_uppers = [], [], []
        aCFR_means, aCFR_lowers, aCFR_uppers = [], [], []
        ITS_factual_means, ITS_factual_lowers, ITS_factual_uppers = [], [], []
        ITS_cf_means, ITS_cf_lowers, ITS_cf_uppers = [], [], []

        for mc_run_idx in scen_df_valid["mc_run"].astype(int) - 1:
            # Load posterior summary for the superior sCFR-O model for plotting
            posterior_summary = results_io.load_posterior_summary_for_run(scenario_id, mc_run_idx, config.OUTPUT_DIR_POSTERIOR_SUMMARIES)
            if posterior_summary:
                sCFR_means.append(posterior_summary.get("p_mean", []))
                sCFR_lowers.append(posterior_summary.get("p_q025", []))
                sCFR_uppers.append(posterior_summary.get("p_q975", []))
                sCFR_cf_means.append(posterior_summary.get("p_cf_mean", []))
                sCFR_cf_lowers.append(posterior_summary.get("p_cf_q025", []))
                sCFR_cf_uppers.append(posterior_summary.get("p_cf_q975", []))

            # Regenerate data to calculate benchmarks for this run
            benchmark_results = results_io.load_benchmark_results(
                scenario_id, mc_run_idx, config.OUTPUT_DIR_BENCHMARK_RESULTS
            )
            if benchmark_results:
                cCFR_means.append(benchmark_results.get("cCFR_cumulative", []))
                cCFR_lowers.append(benchmark_results.get("cCFR_cumulative_lower", []))
                cCFR_uppers.append(benchmark_results.get("cCFR_cumulative_upper", []))
                
                aCFR_means.append(benchmark_results.get("aCFR_cumulative", []))
                aCFR_lowers.append(benchmark_results.get("aCFR_cumulative_lower", []))
                aCFR_uppers.append(benchmark_results.get("aCFR_cumulative_upper", []))

                ITS_factual_means.append(benchmark_results.get("its_factual_mean",[]))
                ITS_factual_lowers.append(benchmark_results.get("its_factual_lower",[]))
                ITS_factual_uppers.append(benchmark_results.get("its_factual_upper",[]))
                ITS_cf_means.append(benchmark_results.get("its_counterfactual_mean",[]))
                ITS_cf_lowers.append(benchmark_results.get("its_counterfactual_lower",[]))
                ITS_cf_uppers.append(benchmark_results.get("its_counterfactual_upper",[]))

        # Calculate the point-wise average of the curves and intervals
        agg_plot_dict = {
            "scenario_id": scenario_id,
            "true_r_t": sim_data_true["true_r_0_t"][:T_analyze],
            "true_rcf_0_t": sim_data_true["true_rcf_0_t"][:T_analyze],
            "estimated_r_t_dict": {
                "sCFR": {
                    "mean": np.mean([s for s in sCFR_means if len(s)>0], axis=0)[:T_analyze],
                    "lower": np.mean([s for s in sCFR_lowers if len(s)>0], axis=0)[:T_analyze],
                    "upper": np.mean([s for s in sCFR_uppers if len(s)>0], axis=0)[:T_analyze],
                    "cf_mean": np.mean([s for s in sCFR_cf_means if len(s)>0], axis=0)[:T_analyze],
                    "cf_lower": np.mean([s for s in sCFR_cf_lowers if len(s)>0], axis=0)[:T_analyze],
                    "cf_upper": np.mean([s for s in sCFR_cf_uppers if len(s)>0], axis=0)[:T_analyze]
                },
                "cCFR_cumulative": {
                    "mean": np.mean(cCFR_means, axis=0)[:T_analyze],
                    "lower": np.mean(cCFR_lowers, axis=0)[:T_analyze],
                    "upper": np.mean(cCFR_uppers, axis=0)[:T_analyze]
                },
                "aCFR_cumulative": {
                    "mean": np.mean(aCFR_means, axis=0)[:T_analyze],
                    "lower": np.mean(aCFR_lowers, axis=0)[:T_analyze],
                    "upper": np.mean(aCFR_uppers, axis=0)[:T_analyze]
                },
                "ITS_MLE": {
                    "factual_mean": np.mean(ITS_factual_means, axis=0)[:T_analyze],
                    "factual_lower": np.mean(ITS_factual_lowers, axis=0)[:T_analyze],
                    "factual_upper": np.mean(ITS_factual_uppers, axis=0)[:T_analyze],
                    "cf_mean": np.mean(ITS_cf_means, axis=0)[:T_analyze],
                    "cf_lower": np.mean(ITS_cf_lowers, axis=0)[:T_analyze],
                    "cf_upper": np.mean(ITS_cf_uppers, axis=0)[:T_analyze]
                }
            }
        }
        aggregated_plot_data_list.append(agg_plot_dict)

    return aggregated_plot_data_list

def main_analysis():
    """Main function to orchestrate the post-hoc analysis of simulation results."""
    all_loaded_metrics_list = []
    
    print("Starting analysis of existing simulation results...")
    
    # Create output directories if they don't exist
    for dir_path in [config.OUTPUT_DIR_PLOTS, config.OUTPUT_DIR_TABLES, config.OUTPUT_DIR_RESULTS_CSV]:
        os.makedirs(dir_path, exist_ok=True)

    # --- Load all saved metrics ---
    for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Loading All Metrics")):
        scenario_id = scenario_config_dict["id"]
        for mc_run_idx in range(config.NUM_MONTE_CARLO_RUNS):
            run_metrics = results_io.load_run_metrics(scenario_id, mc_run_idx, config.OUTPUT_DIR_RUN_METRICS_JSON)
            if run_metrics:
                all_loaded_metrics_list.append(run_metrics)
    
    if not all_loaded_metrics_list:
        print("No metrics files found. Cannot generate plots or tables.")
        return
        
    results_df_all = pd.DataFrame(all_loaded_metrics_list)
    results_df_valid = sanitize_metrics_dataframe(results_df_all)
    results_df_valid = results_df_valid[results_df_valid['error'].isin([None, "None"])].copy()
    
    if results_df_valid.empty:
        print("No valid simulation runs found. Analysis cannot proceed.")
        return
    
    # --- Prepare data for plotting and tables ---
    cover_cols = [col for col in results_df_valid.columns if 'cover' in col]
    for col in cover_cols:
        results_df_valid[col] = results_df_valid[col].astype('Int64')

    summary_metrics_mean = results_df_valid.groupby("scenario_id").mean(numeric_only=True).reset_index()
    summary_metrics_std = results_df_valid.groupby("scenario_id").std(numeric_only=True).reset_index()
    
    summary_metrics_mean = summary_metrics_mean.add_suffix('_mean').rename(columns={'scenario_id_mean':'scenario_id'})
    summary_metrics_std = summary_metrics_std.add_suffix('_std').rename(columns={'scenario_id_std':'scenario_id'})
    results_df_summary_for_tables = pd.merge(summary_metrics_mean, summary_metrics_std, on="scenario_id", how="left")
    
    analysis_csv_path = os.path.join(config.OUTPUT_DIR_RESULTS_CSV, "all_scenarios_metrics_aggregated.csv")
    results_df_summary_for_tables.to_csv(analysis_csv_path, index=False)
    print(f"\nAggregated summary metrics saved to {analysis_csv_path}")

    # --- Generate Plots and Tables ---
    print("\nPreparing aggregated data for summary plot...")
    aggregated_plot_data = prepare_aggregated_plot_data(results_df_all)
    
    print("Generating combined 4x3 aggregated summary plot...")
    plotting.plot_aggregated_scenarios_summary(aggregated_plot_data, config.OUTPUT_DIR_PLOTS)
    
    print("Generating summary boxplots...")
    plotting.plot_metric_summary_boxplots(results_df_valid, config.OUTPUT_DIR_PLOTS)
    
    print("Generating LaTeX summary tables...")
    tables.generate_rt_metrics_table(results_df_summary_for_tables, config.OUTPUT_DIR_TABLES)
    tables.generate_param_metrics_table(results_df_summary_for_tables, config.OUTPUT_DIR_TABLES)

    print("\nAnalysis complete.")

In [3]:
# all_loaded_metrics_list = []

# print("Starting analysis of existing simulation results...")

# # Create output directories if they don't exist
# for dir_path in [config.OUTPUT_DIR_PLOTS, config.OUTPUT_DIR_TABLES, config.OUTPUT_DIR_RESULTS_CSV]:
#     os.makedirs(dir_path, exist_ok=True)

# # --- Load all saved metrics ---
# for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Loading All Metrics")):
#     scenario_id = scenario_config_dict["id"]
#     for mc_run_idx in range(config.NUM_MONTE_CARLO_RUNS):
#         run_metrics = results_io.load_run_metrics(scenario_id, mc_run_idx, config.OUTPUT_DIR_RUN_METRICS_JSON)
#         if run_metrics:
#             all_loaded_metrics_list.append(run_metrics)

# results_df_all = pd.DataFrame(all_loaded_metrics_list)
# results_df_valid = sanitize_metrics_dataframe(results_df_all)
# results_df_valid = results_df_valid[results_df_valid['error'].isin([None, "None"])].copy()

In [4]:
# aggregated_plot_data_list = []
# study_global_seed = config.GLOBAL_BASE_SEED

# for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Aggregating Plot Data")):
#     scenario_id = scenario_config_dict["id"]
#     scenario_base_seed = study_global_seed + (scenario_idx * config.NUM_MONTE_CARLO_RUNS * 1000)
    
#     sim_data_true = data_generation.simulate_scenario_data(scenario_config_dict, run_seed=scenario_base_seed)
#     T_analyze = config.T_ANALYSIS_LENGTH

#     scen_df_valid = results_df_all[(results_df_all["scenario_id"] == scenario_id) & (results_df_all["error"].isin([None, "None"]))]
#     if scen_df_valid.empty: continue
    
#     # Stacks for collecting time-series data from all valid runs
#     sCFR_O_means, sCFR_O_lowers, sCFR_O_uppers = [], [], []
#     sCFR_O_cf_means, sCFR_O_cf_lowers, sCFR_O_cf_uppers = [], [], []
#     cCFR_means, cCFR_lowers, cCFR_uppers = [], [], []
#     aCFR_means, aCFR_lowers, aCFR_uppers = [], [], []
#     ITS_factual_means, ITS_factual_lowers, ITS_factual_uppers = [], [], []
#     ITS_cf_means, ITS_cf_lowers, ITS_cf_uppers = [], [], []

#     for mc_run_idx in scen_df_valid["mc_run"].astype(int) - 1:
#         # Load posterior summary for the superior sCFR-O model for plotting
#         posterior_summary = results_io.load_posterior_summary_for_run(scenario_id, mc_run_idx, config.OUTPUT_DIR_POSTERIOR_SUMMARIES, model_name="sCFR_O")
#         if posterior_summary:
#             sCFR_O_means.append(posterior_summary.get("p_mean", []))
#             sCFR_O_lowers.append(posterior_summary.get("p_q025", []))
#             sCFR_O_uppers.append(posterior_summary.get("p_q975", []))
#             sCFR_O_cf_means.append(posterior_summary.get("p_cf_mean", []))
#             sCFR_O_cf_lowers.append(posterior_summary.get("p_cf_q025", []))
#             sCFR_O_cf_uppers.append(posterior_summary.get("p_cf_q975", []))

#         print("sCFR_O:",sCFR_O_means)

#         # Regenerate data to calculate benchmarks for this run
#         sim_data_run = data_generation.simulate_scenario_data(scenario_config_dict, run_seed=(scenario_base_seed + mc_run_idx))
#         benchmark_cis = benchmarks.calculate_benchmark_cis_with_bayesian(sim_data_run["d_t"], sim_data_run["c_t"], sim_data_run["f_s_true"])
#         its_results = benchmarks.calculate_its_with_parametric_bootstrap(
#             sim_data_run["d_t"], sim_data_run["c_t"], sim_data_run["f_s_true"],
#             sim_data_run["Bm_true"], sim_data_run["true_intervention_times_0_abs"], sim_data_run["beta_signs_true"]
#         )
        
#         cCFR_means.append(benchmarks.calculate_crude_cfr(sim_data_run["d_t"], sim_data_run["c_t"], cumulative=True))
#         cCFR_lowers.append(benchmark_cis["cCFR_cumulative_lower"])
#         cCFR_uppers.append(benchmark_cis["cCFR_cumulative_upper"])
        
#         aCFR_means.append(benchmarks.calculate_nishiura_cfr_cumulative(sim_data_run["d_t"], sim_data_run["c_t"], sim_data_run["f_s_true"]))
#         aCFR_lowers.append(benchmark_cis["aCFR_cumulative_lower"])
#         aCFR_uppers.append(benchmark_cis["aCFR_cumulative_upper"])
        
#         ITS_factual_means.append(its_results["its_factual_mean"])
#         ITS_factual_lowers.append(its_results["its_factual_lower"])
#         ITS_factual_uppers.append(its_results["its_factual_upper"])
#         ITS_cf_means.append(its_results["its_counterfactual_mean"])
#         ITS_cf_lowers.append(its_results["its_counterfactual_lower"])
#         ITS_cf_uppers.append(its_results["its_counterfactual_upper"])

#     # Calculate the point-wise average of the curves and intervals
#     agg_plot_dict = {
#         "scenario_id": scenario_id,
#         "true_r_t": sim_data_true["true_r_0_t"][:T_analyze],
#         "true_rcf_0_t": sim_data_true["true_rcf_0_t"][:T_analyze],
#         "estimated_r_t_dict": {
#             "sCFR-O": {
#                 "mean": np.mean([s for s in sCFR_O_means if len(s)>0], axis=0)[:T_analyze],
#                 "lower": np.mean([s for s in sCFR_O_lowers if len(s)>0], axis=0)[:T_analyze],
#                 "upper": np.mean([s for s in sCFR_O_uppers if len(s)>0], axis=0)[:T_analyze],
#                 "cf_mean": np.mean([s for s in sCFR_O_cf_means if len(s)>0], axis=0)[:T_analyze],
#                 "cf_lower": np.mean([s for s in sCFR_O_cf_lowers if len(s)>0], axis=0)[:T_analyze],
#                 "cf_upper": np.mean([s for s in sCFR_O_cf_uppers if len(s)>0], axis=0)[:T_analyze]
#             },
#             "cCFR_cumulative": {
#                 "mean": np.mean(cCFR_means, axis=0)[:T_analyze],
#                 "lower": np.mean(cCFR_lowers, axis=0)[:T_analyze],
#                 "upper": np.mean(cCFR_uppers, axis=0)[:T_analyze]
#             },
#             "aCFR_cumulative": {
#                 "mean": np.mean(aCFR_means, axis=0)[:T_analyze],
#                 "lower": np.mean(aCFR_lowers, axis=0)[:T_analyze],
#                 "upper": np.mean(aCFR_uppers, axis=0)[:T_analyze]
#             },
#             "ITS_MLE": {
#                 "factual_mean": np.mean(ITS_means, axis=0)[:T_analyze],
#                 "factual_lower": np.mean(ITS_lowers, axis=0)[:T_analyze],
#                 "factual_upper": np.mean(ITS_uppers, axis=0)[:T_analyze],
#                 "cf_mean": np.mean(ITS_cf_means, axis=0)[:T_analyze],
#                 "cf_lower": np.mean(ITS_cf_lowers, axis=0)[:T_analyze],
#                 "cf_upper": np.mean(ITS_cf_uppers, axis=0)[:T_analyze]
#             }
#         }
#     }
#     aggregated_plot_data_list.append(agg_plot_dict)

In [5]:
# """Main function to orchestrate the post-hoc analysis of simulation results."""
# all_loaded_metrics_list = []

# print("Starting analysis of existing simulation results...")

# # Create output directories if they don't exist
# for dir_path in [config.OUTPUT_DIR_PLOTS, config.OUTPUT_DIR_TABLES, config.OUTPUT_DIR_RESULTS_CSV]:
#     os.makedirs(dir_path, exist_ok=True)

# # --- Load all saved metrics ---
# for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Loading All Metrics")):
#     scenario_id = scenario_config_dict["id"]
#     for mc_run_idx in range(config.NUM_MONTE_CARLO_RUNS):
#         run_metrics = results_io.load_run_metrics(scenario_id, mc_run_idx, config.OUTPUT_DIR_RUN_METRICS_JSON)
#         if run_metrics:
#             all_loaded_metrics_list.append(run_metrics)

# results_df_all = pd.DataFrame(all_loaded_metrics_list)
# results_df_valid = sanitize_metrics_dataframe(results_df_all)
# results_df_valid = results_df_valid[results_df_valid['error'].isin([None, "None"])].copy()

# # --- Prepare data for plotting and tables ---
# cover_cols = [col for col in results_df_valid.columns if 'cover' in col]
# for col in cover_cols:
#     results_df_valid[col] = results_df_valid[col].astype('Int64')

# summary_metrics_mean = results_df_valid.groupby("scenario_id").mean(numeric_only=True).reset_index()
# summary_metrics_std = results_df_valid.groupby("scenario_id").std(numeric_only=True).reset_index()

# summary_metrics_mean = summary_metrics_mean.add_suffix('_mean').rename(columns={'scenario_id_mean':'scenario_id'})
# summary_metrics_std = summary_metrics_std.add_suffix('_std').rename(columns={'scenario_id_std':'scenario_id'})
# results_df_summary_for_tables = pd.merge(summary_metrics_mean, summary_metrics_std, on="scenario_id", how="left")

# analysis_csv_path = os.path.join(config.OUTPUT_DIR_RESULTS_CSV, "all_scenarios_metrics_aggregated.csv")
# results_df_summary_for_tables.to_csv(analysis_csv_path, index=False)
# print(f"\nAggregated summary metrics saved to {analysis_csv_path}")

# # --- Generate Plots and Tables ---
# print("\nPreparing aggregated data for summary plot...")
# aggregated_plot_data = prepare_aggregated_plot_data(results_df_all)

In [6]:
# plotting.plot_aggregated_scenarios_summary(aggregated_plot_data, config.OUTPUT_DIR_PLOTS)

In [7]:
# """Main function to orchestrate the post-hoc analysis of simulation results."""
# all_loaded_metrics_list = []

# print("Starting analysis of existing simulation results...")

# # Create output directories if they don't exist
# for dir_path in [config.OUTPUT_DIR_PLOTS, config.OUTPUT_DIR_TABLES, config.OUTPUT_DIR_RESULTS_CSV]:
#     os.makedirs(dir_path, exist_ok=True)

# # --- Load all saved metrics ---
# for scenario_idx, scenario_config_dict in enumerate(tqdm(config.SCENARIOS, desc="Loading All Metrics")):
#     scenario_id = scenario_config_dict["id"]
#     for mc_run_idx in range(config.NUM_MONTE_CARLO_RUNS):
#         run_metrics = results_io.load_run_metrics(scenario_id, mc_run_idx, config.OUTPUT_DIR_RUN_METRICS_JSON)
#         if run_metrics:
#             all_loaded_metrics_list.append(run_metrics)

# results_df_all = pd.DataFrame(all_loaded_metrics_list)
# results_df_valid = sanitize_metrics_dataframe(results_df_all)
# results_df_valid = results_df_valid[results_df_valid['error'].isin([None, "None"])].copy()

# # --- Prepare data for plotting and tables ---
# cover_cols = [col for col in results_df_valid.columns if 'cover' in col]
# for col in cover_cols:
#     results_df_valid[col] = results_df_valid[col].astype('Int64')

# summary_metrics_mean = results_df_valid.groupby("scenario_id").mean(numeric_only=True).reset_index()
# summary_metrics_std = results_df_valid.groupby("scenario_id").std(numeric_only=True).reset_index()

# summary_metrics_mean = summary_metrics_mean.add_suffix('_mean').rename(columns={'scenario_id_mean':'scenario_id'})
# summary_metrics_std = summary_metrics_std.add_suffix('_std').rename(columns={'scenario_id_std':'scenario_id'})
# results_df_summary_for_tables = pd.merge(summary_metrics_mean, summary_metrics_std, on="scenario_id", how="left")

# analysis_csv_path = os.path.join(config.OUTPUT_DIR_RESULTS_CSV, "all_scenarios_metrics_aggregated.csv")
# results_df_summary_for_tables.to_csv(analysis_csv_path, index=False)
# print(f"\nAggregated summary metrics saved to {analysis_csv_path}")

# # --- Generate Plots and Tables ---
# print("\nPreparing aggregated data for summary plot...")
# aggregated_plot_data = prepare_aggregated_plot_data(results_df_all)

# print("Generating combined 4x3 aggregated summary plot...")
# plotting.plot_aggregated_scenarios_summary(aggregated_plot_data, config.OUTPUT_DIR_PLOTS)

# print("Generating summary boxplots...")
# plotting.plot_metric_summary_boxplots(results_df_valid, config.OUTPUT_DIR_PLOTS)

In [8]:
# results_df_summary_for_tables

In [9]:
# tables.generate_rt_metrics_table(results_df_summary_for_tables, config.OUTPUT_DIR_TABLES)

## 3. Execute Analysis

Run the cell below to perform the analysis. Make sure the simulation output directories in `config.py` point to where your simulation results are stored.

In [10]:
if __name__ == '__main__':
    main_analysis()

Starting analysis of existing simulation results...


Loading All Metrics:   0%|          | 0/12 [00:00<?, ?it/s]


Aggregated summary metrics saved to ./simulation_outputs/results_csv/all_scenarios_metrics_aggregated.csv

Preparing aggregated data for summary plot...


Aggregating Plot Data:   0%|          | 0/12 [00:00<?, ?it/s]

Generating combined 4x3 aggregated summary plot...
Generating summary boxplots...
Generating LaTeX summary tables...
Summary table for r_t metrics saved to ./simulation_outputs/tables/summary_table_rt_metrics.csv
LaTeX table for r_t metrics saved to ./simulation_outputs/tables/summary_table_rt_metrics.tex
Summary table for sCFR parameter metrics saved to ./simulation_outputs/tables/summary_table_param_metrics_sCFR.csv
LaTeX table for sCFR parameter metrics saved to ./simulation_outputs/tables/summary_table_param_metrics_sCFR.tex
Summary table for ITS parameter metrics saved to ./simulation_outputs/tables/summary_table_param_metrics_ITS.csv
LaTeX table for ITS parameter metrics saved to ./simulation_outputs/tables/summary_table_param_metrics_ITS.tex

Analysis complete.
